# Heart Disease Prediction - SVC model

In [9]:
import numpy as np
import pandas as pd

In [10]:
dataset = pd.read_csv('heart.csv')

In [11]:
dataset.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [12]:
dataset.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [13]:
categorical = ['sex', 'cp', 'restecg', 'slp', 'thall']
do_not_touch = ['fbs', 'exng']
non_categorical = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak', 'caa']

### OneHotEncoding categorical values

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [15]:
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),categorical)],remainder='passthrough')
X = ct.fit_transform(dataset[categorical+do_not_touch+non_categorical])
y = dataset['output'].values

In [16]:
X[0,:]

array([  0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,   0. ,
         1. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
        63. , 145. , 233. , 150. ,   2.3,   0. ])

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=1)

#### Last 6 columns of the dataset are non categorical values. So they need to be scaled.

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [19]:
X_train[:,-6:] = scaler.fit_transform(X_train[:,-6:])
X_test[:,-6:] = scaler.transform(X_test[:,-6:])

In [20]:
X_train[0,:]

array([ 0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        , -0.61990074, -0.08877873,
        0.37511601,  0.91545786, -0.37805012, -0.70686683])

In [21]:
from sklearn.svm import SVC
estimator = SVC()

parameters = [{'kernel':['rbf'],
               'C':[1,10,100,1000],
               'gamma':[1,0.1,0.001,0.0001],
            },
            {'kernel':['poly'],
               'C':[1,10,100,1000],
               'gamma':[1,0.1,0.001,0.0001],
             'degree':range(1,5)}
             ]

### Using Grid Search to find best fit SVC model

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [24]:
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

Fitting 10 folds for each of 80 candidates, totalling 800 fits


SVC(C=100, gamma=0.0001)

In [25]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[13  3]
 [ 0 15]]


0.9032258064516129

### So, this is the final confusion matrix and the accuracy score